In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb # Import XGBoost

# Directory for saving outputs
output_result = 'xgb_polygon' # Changed output directory for XGBoost

# Ensure directories exist
os.makedirs(output_result, exist_ok=True)

df = pd.read_csv("21.5.2025.csv")
df = df[['timestamp', 'x_snap', 'y_snap']].dropna()
df = df.sort_values(by='timestamp').reset_index(drop=True)

def iqr_filter(series):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    return series.between(lower, upper)

mask = iqr_filter(df['x_snap']) & iqr_filter(df['y_snap'])
df = df[mask].reset_index(drop=True)

def kalman_filter_1d(data, process_var=1e-3, meas_var=0.108**2):
    n = len(data)
    xhat = np.zeros(n)
    P = np.zeros(n)
    xhat[0] = data[0]
    P[0] = 1.0
    for k in range(1, n):
        xhat[k] = xhat[k-1]
        P[k] = P[k-1] + process_var
        K = P[k] / (P[k] + meas_var)
        xhat[k] = xhat[k] + K * (data[k] - xhat[k])
        P[k] = (1 - K) * P[k]
    return xhat

df['x_kalman'] = kalman_filter_1d(df['x_snap'].values)
df['y_kalman'] = kalman_filter_1d(df['y_snap'].values)

# --- Remove the last 8 points from the history trajectory ---
# df = df.iloc[:-8].reset_index(drop=True) #

# === NEW CODE FOR POLYGON FILTERING ===
# Define the polygon vertices from your JSON
polygon_json = "{\"l1\":[[27.05,12.7,0],[81.19,12.7,0]],\"l2\":[[81.19,12.7,0],[81.19,28.87,0]],\"l3\":[[81.19,28.87,0],[27.05,28.87,0]],\"l4\":[[27.05,28.87,0],[27.05,12.7,0]]}"
polygon = json.loads(polygon_json)

# Extract min/max x and y from the polygon to define a bounding box
min_x_poly = min(line[0][0] for line in polygon.values())
max_x_poly = max(line[0][0] for line in polygon.values())
min_y_poly = min(line[0][1] for line in polygon.values())
max_y_poly = max(line[0][1] for line in polygon.values())

# Filter DataFrame based on the polygon boundaries
df_filtered_polygon = df[
    (df['x_kalman'] >= min_x_poly) & (df['x_kalman'] <= max_x_poly) &
    (df['y_kalman'] >= min_y_poly) & (df['y_kalman'] <= max_y_poly)
].reset_index(drop=True)

# === Create sliding window input features ===
window_size = 15

# Use the polygon-filtered DataFrame here
X, y = [], []
for i in range(window_size, len(df_filtered_polygon) - 1):
    window = df_filtered_polygon[['x_kalman', 'y_kalman']].iloc[i-window_size:i].values.flatten()
    next_pos = df_filtered_polygon[['x_kalman', 'y_kalman']].iloc[i + 1].values
    X.append(window)
    y.append(next_pos)

X = np.array(X)
y = np.array(y)

X_trainval, X_test_gt, y_trainval, y_test_gt = train_test_split(X, y, test_size=0.3, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.125, random_state=42)
X_test, X_gt, y_test, y_gt = train_test_split(X_test_gt, y_test_gt, test_size=0.5, random_state=42)

# --- XGBoost Model ---
model = xgb.XGBRegressor(
    n_estimators=100,      # Number of boosting rounds (trees)
    learning_rate=0.15,     # Step size shrinkage
    max_depth=5,           # Maximum depth of a tree
    random_state=42,
    objective='reg:squarederror' # Objective function for regression
)

# Fit Model
model.fit(X_train, y_train)

# Predict
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# Compute RMSE
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"Train RMSE: {train_rmse:.4f}")
print(f"Validation RMSE: {val_rmse:.4f}")
print(f"Test RMSE: {test_rmse:.4f}")

def predict_recursive(model, start_window, n_steps=20):
    preds = []
    window = start_window.copy()
    for _ in range(n_steps):
        # XGBoost predict function also expects 2D input
        pred = model.predict(window.reshape(1, -1))[0]
        preds.append(pred)
        window = np.roll(window, -2)
        window[-2:] = pred
    return np.array(preds)

start_window = X_gt[0]
n_steps=10
pred_path = predict_recursive(model, start_window, n_steps=n_steps)

# Locate index in DataFrame
start_x, start_y = start_window[-2], start_window[-1]

# IMPORTANT: Use df_filtered_polygon to find the start_idx for the true_gt_path
# This ensures consistency with the filtered data used for X and y
dists = np.sqrt((df_filtered_polygon['x_kalman'] - start_x)**2 + (df_filtered_polygon['y_kalman'] - start_y)**2)
start_idx = dists.idxmin()

true_gt_path = df_filtered_polygon[['x_kalman', 'y_kalman']].iloc[start_idx + 1: start_idx + 1 + n_steps].values

def euclidean(p1, p2):
    return np.sqrt((p1[0] - p2[0])**2 + (p1[1] - p2[1])**2)

step_errors = [euclidean(p, g) for p, g in zip(pred_path, true_gt_path)]
avg_error =  np.mean(step_errors)

# Plot
plt.figure(figsize=(7, 4))
plt.plot(range(1, len(step_errors)+1), step_errors, marker='o', color='purple')
plt.axhline(4.0, linestyle='--', color='red', label="4m Threshold")
plt.title(f"Step-wise Average Euclidean Distance Error: {avg_error:.4f} meters")
plt.xlabel("Step")
plt.ylabel("Distance Error")
plt.grid(True)
plt.savefig(os.path.join(output_result, "stepwise_error_plot.png")) # Use os.path.join for paths
plt.show()

# CSV
rows = [{
    'step': i+1,
    'gt_x': gt[0],
    'gt_y': gt[1],
    'pred_x': pr[0],
    'pred_y': pr[1],
    'euclidean_distance': dist
} for i, (gt, pr, dist) in enumerate(zip(true_gt_path, pred_path, step_errors))]

df_error = pd.DataFrame(rows)
df_error.to_csv(os.path.join(output_result, "stepwise_prediction_vs_gt.csv"), index=False) # Use os.path.join for paths
print(f"✅ Exported: {os.path.join(output_result, 'stepwise_prediction_vs_gt.csv')} - Average Error: {avg_error:.4f}")

# === Full Combined Plot: History, Predicted Path, Ground Truth ===
plt.figure(figsize=(8, 6))

# Reference polygon lines
def draw_ref_lines_plot(ax_plot):
    for line in polygon.values():
        x = [p[0] for p in line]
        y = [p[1] for p in line]
        ax_plot.plot(x, y, 'k--', linewidth=1)

draw_ref_lines_plot(plt.gca())

# Raw history (all past data) - using the polygon-filtered data
plt.plot(df_filtered_polygon['x_kalman'], df_filtered_polygon['y_kalman'], 'gray', alpha=0.5, label='Filtered History')
plt.scatter(df_filtered_polygon['x_kalman'].iloc[0], df_filtered_polygon['y_kalman'].iloc[0], s=100, color='blue', marker='s', label='start')
plt.scatter(df_filtered_polygon['x_kalman'].iloc[-1], df_filtered_polygon['y_kalman'].iloc[-1], s=100, color='red', marker='s', label='end')

# Ground Truth path (green)
plt.plot(true_gt_path[:, 0], true_gt_path[:, 1], 'g-o', label='Ground Truth')

# Predicted path (blue dashed)
plt.plot(pred_path[:, 0], pred_path[:, 1], 'b--o', label='Predicted Path')

plt.title("Trajectory Overview: Filtered History, Ground Truth, and Prediction (XGBoost)")
plt.xlabel("x")
plt.ylabel("y")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(output_result, "full_trajectory_overview.png")) # Use os.path.join for paths
plt.show()